# RAG: Fundamentos e técnicas avançadas

 > Aviso: Este post foi traduzido para o português usando um modelo de tradução automática. Por favor, me avise se encontrar algum erro.

Neste post, vamos ver em que consiste a técnica de `RAG` (`Retrieval Augmented Generation`) e como pode ser implementada em um modelo de linguagem.

Para que seja gratuito, em vez de usar uma conta da OpenAI (como você verá na maioria dos tutoriais), vamos usar a `API inference` da Hugging Face, que tem um free tier de 1000 requisições por dia, que para fazer este post é mais do que suficiente.

## Configuração da `API Inference` de Hugging Face

Para poder usar a `API Inference` de HuggingFace, a primeira coisa que você precisa é ter uma conta no HuggingFace. Uma vez que você a tenha, precisa ir a [Access tokens](https://huggingface.co/settings/keys) na configuração do seu perfil e gerar um novo token.

Temos que dar um nome, no meu caso vou chamar `rag-fundamentals` e habilitar a permissão `Make calls to serverless Inference API`. Um token será criado para nós, que precisamos copiar.

Para gerenciar o token vamos criar um arquivo na mesma rota em que estamos trabalhando chamado `.env` e vamos colocar o token que copiamos no arquivo da seguinte maneira:

```bash
RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN="hf_...."
```

Agora, para obter o token, precisamos ter o `dotenv` instalado, o que fazemos através

```bash
pip install python-dotenv
```

e executamos o seguinte

In [1]:
import os
import dotenv

dotenv.load_dotenv()

RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN = os.getenv("RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN")

Agora que temos um token, criamos um cliente, para isso precisamos ter instalada a biblioteca `huggingface_hub`, que fazemos usando conda ou pip

```bash
conda install -c conda-forge huggingface_hub
```

o

```bash
pip install --upgrade huggingface_hub
```


Agora temos que escolher qual modelo vamos usar. Você pode ver os modelos disponíveis na página de [Supported models](https://huggingface.co/docs/api-inference/supported-models) da documentação da `API Inference` da Hugging Face.

Como no momento de escrever a postagem, o melhor disponível é `Qwen2.5-72B-Instruct`, vamos usar esse modelo.

In [2]:
MODEL = "Qwen/Qwen2.5-72B-Instruct"

Agora podemos criar o cliente

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN, model=MODEL)
client

<InferenceClient(model='Qwen/Qwen2.5-72B-Instruct', timeout=None)>

Estamos fazendo um teste para ver se funciona

In [4]:
message = [
	{ "role": "user", "content": "Hola, qué tal?" }
]

stream = client.chat.completions.create(
	messages=message, 
	temperature=0.5,
	max_tokens=1024,
	top_p=0.7,
	stream=False
)

response = stream.choices[0].message.content
print(response)

¡Hola! Estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


## O que é `RAG`?

`RAG` são as siglas de `Retrieval Augmented Generation`, é uma técnica criada para obter informações de documentos. Embora os LLMs possam ser muito poderosos e ter muito conhecimento, nunca serão capazes de responder sobre documentos privados, como relatórios da sua empresa, documentação interna, etc. Por isso foi criado o `RAG`, para poder usar esses LLMs nessa documentação privada.

![O que é RAG?](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/RAG.webp)

A ideia consiste em que um usuário faz uma pergunta sobre essa documentação privada, o sistema é capaz de obter a parte da documentação na qual está a resposta a essa pergunta, passa-se a pergunta e a parte da documentação para um LLM e o LLM gera a resposta para o usuário.

### Como a informação é armazenada?

É sabido, e se você não sabia eu te conto agora, que os LLMs têm um limite de informações que podem ser passadas a eles, isso é chamado de janela de contexto. Isso se deve às arquiteturas internas dos LLMs que agora não vêm ao caso. Mas o importante é que não se pode passar um documento e uma pergunta sem mais, porque é provável que o LLM não seja capaz de processar todas essas informações.

Nos casos em que geralmente se passa mais informações do que a janela de contexto permite, o que geralmente acontece é que o LLM não presta atenção ao final da entrada. Imagine que você pergunte ao LLM sobre algo do seu documento, que essa informação esteja no final do documento e o LLM não a leia.

Por isso, o que se faz é dividir a documentação em blocos chamados de `chunk`s. Dessa forma, a documentação é armazenada em um monte de `chunk`s, que são pedaços dessa documentação. Assim, quando o usuário faz uma pergunta, é passado para o LLM o `chunk` em que está a resposta para essa pergunta.

Além de dividir a documentação em `chunk`s, estes são convertidos em embeddings, que são representações numéricas dos `chunk`s. Isso é feito porque os LLMs na verdade não entendem texto, mas sim números, e os `chunk`s são convertidos em números para que o LLM possa entendê-los. Se quiser entender mais sobre os embeddings, pode ler meu post sobre [transformers](https://www.maximofn.com/transformers) no qual explico como funcionam os transformers, que é a arquitetura por trás dos LLMs. Você também pode ler meu post sobre [ChromaDB](https://www.maximofn.com/chromadb) onde explico como os embeddings são armazenados em um banco de dados vetorial. E além disso, seria interessante que lesse meu post sobre a biblioteca [HuggingFace Tokenizers](https://www.maximofn.com/hugging-face-tokenizers) no qual se explica como o texto é tokenizado, que é a etapa anterior à geração dos embeddings.

![RAG - embeddings](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/RAG-embeddings.webp)

### Como obter o `chunk` correto?

Dissemos que a documentação é dividida em `chunks` e o `chunk` em que está a resposta à pergunta do usuário é passado ao LLM. Mas, como se sabe em qual `chunk` está a resposta? Para isso, o que se faz é converter a pergunta do usuário em um embedding, e calcula-se a similaridade entre o embedding da pergunta e os embeddings dos `chunks`. Dessa forma, o `chunk` com maior similaridade é o que é passado ao LLM.

![RAG - similaridade de embeddings](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/rag-chunk_retreival.webp)

### Vamos ver o que é `RAG`

Por um lado temos o `retrieval`, que é obter o `chunk` correto da documentação, por outro lado temos o `augmented`, que é passar para o LLM a pergunta do usuário e o `chunk`, e por último temos o `generation`, que é obter a resposta gerada pelo LLM.

## Banco de dados vetorial

Vimos que a documentação é dividida em `chunk`s e armazenada em um banco de dados vetorial, portanto, precisamos usar um. Para este post, vou usar o [ChromaDB](https://www.trychroma.com/), que é um banco de dados vetorial bastante utilizado e, além disso, tenho um [post](https://www.maximofn.com/chromadb) no qual explico como funciona.

Portanto, primeiro precisamos instalar a biblioteca do ChromaDB, para isso a instalamos com Conda ou com Pip

```bash
conda install conda-forge::chromadb
```

Então, me envie o texto markdown que você gostaria que eu traduza para o português.

```bash
pip install chromadb
```

### Função de embedding

Como dissemos, tudo se baseará em embeddings, por isso a primeira coisa que fazemos é criar uma função para obter embeddings de um texto. Vamos usar o modelo `sentence-transformers/all-MiniLM-L6-v2`

In [5]:
import chromadb.utils.embedding_functions as embedding_functions

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
      
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN,
    model_name=EMBEDDING_MODEL
)

Testamos a função de embedding

In [6]:
embedding = huggingface_ef(["Hello, how are you?",])
embedding[0].shape

(384,)

Obtemos um embedding de dimensão 384. Embora a missão deste post não seja explicar os embeddings, em resumo, nossa função de embedding categorizou a frase `Hello, how are you?` em um espaço de 384 dimensões.

### Cliente ChromaDB

Agora que temos nossa função de embedding, podemos criar um cliente do ChromaDB.

Primeiro criamos uma pasta onde será guardado o banco de dados vetorial

In [7]:
from pathlib import Path
      
chroma_path = Path("chromadb_persisten_storage")
chroma_path.mkdir(exist_ok=True)

Agora criamos o cliente

In [8]:
from chromadb import PersistentClient

chroma_client = PersistentClient(path = str(chroma_path))

### Coleção

Quando temos o cliente do ChromaDB, o próximo passo é criar uma coleção. Uma coleção é um conjunto de vetores, no nosso caso os `chunks` da documentação.

Criamos isso indicando a função de embedding que vamos usar

In [9]:
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(name=collection_name, embedding_function=huggingface_ef)

## Carregamento de documentos

Agora que criamos o banco de dados vetorial, precisamos dividir a documentação em `chunk`s e guardá-los no banco de dados vetorial.

### Função de carregamento de documentos

Primeiro criamos uma função para carregar todos os documentos `.txt` de um diretório

In [10]:
def load_one_document_from_directory(directory, file):
    with open(os.path.join(directory, file), "r") as f:
        return {"id": file, "text": f.read()}

def load_documents_from_directory(directory):
    documents = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            documents.append(load_one_document_from_directory(directory, file))
    return documents


### Função para dividir a documentação em `chunk`s

Uma vez que temos os documentos, nós os dividimos em `chunks`

In [11]:
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks


### Função para gerar embeddings de um `chunk`

Agora que temos os `chunks`, geramos os embeddings de cada um deles

Depois veremos por quê, mas para gerar os embeddings vamos fazê-lo de maneira local e não através da API do Hugging Face. Para isso, precisamos ter instalado o [PyTorch](https://pytorch.org) e `sentence-transformers`, para isso fazemos

``` bash
```markdown
pip install -U sentence-transformers
```
```

In [12]:
from sentence_transformers import SentenceTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_model = SentenceTransformer(EMBEDDING_MODEL).to(device)

def get_embeddings(text):
    try:
        embedding = embedding_model.encode(text, device=device)
        return embedding
    except Exception as e:
        print(f"Error: {e}")
        exit(1)

Vamos testar agora esta função de embeddings localmente

In [13]:
text = "Hello, how are you?"
embedding = get_embeddings(text)
embedding.shape

(384,)

Vemos que obtemos um embedding da mesma dimensão que quando o fazíamos com a API de Hugging Face

O modelo `sentence-transformers/all-MiniLM-L6-v2` tem apenas 22M de parâmetros, então você vai poder executá-lo em qualquer GPU. Mesmo se você não tiver GPU, será capaz de executá-lo em uma CPU.

O LLM que vamos usar para gerar as respostas, que é o `Qwen2.5-72B-Instruct`, como o próprio nome indica, é um modelo de 72B de parâmetros, portanto, esse modelo não pode ser executado em qualquer GPU e em uma CPU é impensável de tão lerdo que seria. Por isso, esse LLM será usado através da API, mas no momento de gerar os embeddings, podemos fazê-lo localmente sem problemas.

### Documentos com os quais vamos testar

Para fazer todos esses testes, eu baixei o dataset [aws-case-studies-and-blogs](https://www.kaggle.com/datasets/harshsinghal/aws-case-studies-and-blogs) e o deixei na pasta `rag-txt_dataset`, com os seguintes comandos eu te digo como baixá-lo e descompactá-lo.

Criamos a pasta onde vamos baixar os documentos

In [30]:
!mkdir rag_txt_dataset

Baixamos o `.zip` com os documentos

In [1]:
!curl -L -o ./rag_txt_dataset/archive.zip https://www.kaggle.com/api/v1/datasets/download/harshsinghal/aws-case-studies-and-blogs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1430k  100 1430k    0     0  1082k      0  0:00:01  0:00:01 --:--:-- 2440k


Descompactamos o `.zip`

In [2]:
!unzip rag_txt_dataset/archive.zip -d rag_txt_dataset

Archive:  rag_txt_dataset/archive.zip
  inflating: rag_txt_dataset/23andMe Case Study _ Life Sciences _ AWS.txt  
  inflating: rag_txt_dataset/36 new or updated datasets on the Registry of Open Data_ AI analysis-ready datasets and more _ AWS Public Sector Blog.txt  
  inflating: rag_txt_dataset/54gene _ Case Study _ AWS.txt  
  inflating: rag_txt_dataset/6sense Case Study.txt  
  inflating: rag_txt_dataset/ADP Developed an Innovative and Secure Digital Wallet in a Few Months Using AWS Services _ Case Study _ AWS.txt  
  inflating: rag_txt_dataset/AEON Case Study.txt  
  inflating: rag_txt_dataset/ALTBalaji _ Amazon Web Services.txt  
  inflating: rag_txt_dataset/AWS Case Study - Ineos Team UK.txt  
  inflating: rag_txt_dataset/AWS Case Study - StreamAMG.txt  
  inflating: rag_txt_dataset/AWS Case Study_ Creditsafe.txt  
  inflating: rag_txt_dataset/AWS Case Study_ Immowelt.txt  
  inflating: rag_txt_dataset/AWS Customer Case Study _ Kepler Provides Effective Monitoring of Elderly Care 

Apagamos o `.zip`

In [3]:
!rm rag_txt_dataset/archive.zip

Vemos o que nos sobrou

In [4]:
!ls rag_txt_dataset

'23andMe Case Study _ Life Sciences _ AWS.txt'
'36 new or updated datasets on the Registry of Open Data_ AI analysis-ready datasets and more _ AWS Public Sector Blog.txt'
'54gene _ Case Study _ AWS.txt'
'6sense Case Study.txt'
'Accelerate Time to Business Value Using Amazon SageMaker at Scale with NatWest Group _ Case Study _ AWS.txt'
'Accelerate Your Analytics Journey on AWS with DXC Analytics and AI Platform _ AWS Partner Network (APN) Blog.txt'
'Accelerating customer onboarding using Amazon Connect _ NCS Case Study _ AWS.txt'
'Accelerating Migration at Scale Using AWS Application Migration Service with 3M Company _ Case Study _ AWS.txt'
'Accelerating Time to Market Using AWS and AWS Partner AccelByte _ Omeda Studios Case Study _ AWS.txt'
'Achieving Burstable Scalability and Consistent Uptime Using AWS Lambda with TiVo _ Case Study _ AWS.txt'
'Acrobits Uses Amazon Chime SDK to Easily Create Video Conferencing Application Boosting Collaboration for Global Users _ Acrobits Case Study _

### Criando os `chunk`s!

Listamos os documentos com a função que havíamos criado

In [14]:
dataset_path = "rag_txt_dataset"
documents = load_documents_from_directory(dataset_path)

Comprovamos que o fizemos bem

In [15]:
for document in documents[0:10]:
    print(document["id"])

Run Jobs at Scale While Optimizing for Cost Using Amazon EC2 Spot Instances with ActionIQ _ ActionIQ Case Study _ AWS.txt
Recommend and dynamically filter items based on user context in Amazon Personalize _ AWS Machine Learning Blog.txt
Windsor.txt
Bank of Montreal Case Study _ AWS.txt
The Mill Adventure Case Study.txt
Optimize software development with Amazon CodeWhisperer _ AWS DevOps Blog.txt
Announcing enhanced table extractions with Amazon Textract _ AWS Machine Learning Blog.txt
THREAD _ Life Sciences _ AWS.txt
Deep Pool Optimizes Software Quality Control Using Amazon QuickSight _ Deep Pool Case Study _ AWS.txt
Upstox Saves 1 Million Annually Using Amazon S3 Storage Lens _ Upstox Case Study _ AWS.txt


Agora criamos os `chunk`s.

In [16]:
chunked_documents = []
for document in documents:
    chunks = split_text(document["text"])
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{document['id']}_{i}", "text": chunk})

In [17]:
len(chunked_documents)

3611

Como vemos, existem 3611 `chunk`s. Como o limite diário da API da Hugging Face são 1000 chamadas na conta gratuita, se quisermos criar embeddings de todos os `chunk`s, acabaríamos com as chamadas disponíveis e além disso não poderíamos criar embeddings de todos os `chunk`s.

Voltamos a lembrar, este modelo de embeddings é muito pequeno, apenas 22M de parâmetros, por isso pode ser executado em quase qualquer computador, mais rápido ou mais devagar, mas pode.

Como só vamos criar os embeddings dos `chunk`s uma vez, mesmo que não tenhamos um computador muito potente e demore muito tempo, isso só será executado uma vez. Então, quando quisermos fazer perguntas sobre a documentação, aí sim geraremos os embeddings do prompt com a API do Hugging Face e usaremos o LLM com a API. Portanto, só teremos que passar pelo processo de gerar os embeddings dos `chunk`s uma vez.

Geramos os embeddings dos `chunk`s

Última biblioteca que vamos ter que instalar. Como o processo de gerar os embeddings dos `chunk`s vai ser lento, vamos instalar `tqdm` para que nos mostre uma barra de progresso. Instalamos com conda ou com pip, como preferir.

```bash
```
conda install conda-forge::tqdm
```
```

o

```bash
```
pip install tqdm
```

Geramos os embeddings dos `chunk`s

In [19]:
import tqdm

progress_bar = tqdm.tqdm(chunked_documents)

for chunk in progress_bar:
    embedding = get_embeddings(chunk["text"])
    if embedding is not None:
        chunk["embedding"] = embedding
    else:
        print(f"Error with document {chunk['id']}")

100%|██████████| 3611/3611 [00:16<00:00, 220.75it/s]


Vemos um exemplo

In [60]:
from random import randint

idx = randint(0, len(chunked_documents))
print(f"Chunk id: {chunked_documents[idx]['id']},\n\ntext: {chunked_documents[idx]['text']},\n\nembedding shape: {chunked_documents[idx]['embedding'].shape}")

Chunk id: BNS Group Case Study _ Amazon Web Services.txt_0,

text: Reducing Virtual Machines from 40 to 12
The founders of BNS had been contemplating a migration from the company’s on-premises data center to the public cloud and observed a growing demand for cloud-based operations among current and potential BNS customers.
Français
Configures security according to cloud best practices
Clive Pereira, R&D director at BNS Group, explains, “The database that records Praisal’s SMS traffic resides in Praisal’s AWS environment. Praisal can now run complete analytics across its data and gain insights into what’s happening with its SMS traffic, which is a real game-changer for the organization.”  
Español
 AWS ISV Accelerate Program
 Receiving Strategic, Foundational Support from ISV Specialists
 Learn More
The value that AWS places on the ISV stream sealed the deal in our choice of cloud provider.” 
日本語
  Contact Sales 
BNS is an Australian software provider focused on secure enterprise SMS an

### Carregar os `chunk`s no banco de dados vetorial

Uma vez que temos todos os chunks gerados, os carregamos no banco de dados vetorial. Usamos novamente `tqdm` para que nos mostre uma barra de progresso, porque isso também será lento

In [22]:
import tqdm

progress_bar = tqdm.tqdm(chunked_documents)

for chunk in progress_bar:
    collection.upsert(
        ids=[chunk["id"]],
        documents=chunk["text"],
        embeddings=chunk["embedding"],
    )

100%|██████████| 3611/3611 [00:59<00:00, 60.77it/s]


## Perguntas

Agora que temos o banco de dados vetorial, podemos fazer perguntas à documentação. Para isso, precisamos de uma função que nos devolva o `chunk` correto.

### Obter o `chunk` correto

Agora precisamos de uma função que nos devolva o `chunk` correto, vamos criá-la

In [50]:
def get_top_k_documents(query, k=5):
    results = collection.query(query_texts=query, n_results=k)
    return results

Por último, criamos uma `query`.

Para gerar a query, escolhi aleatoriamente o documento `Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt`, passei-o a um LLM e pedi-lhe que gerasse uma pergunta sobre o documento. A pergunta que ele gerou é

```
How did Neeva use Karpenter and Amazon EC2 Spot Instances to improve its infrastructure management and cost optimization?
```

Assim, obtemos os `chunk`s mais relevantes para essa pergunta

In [51]:
query = "How did Neeva use Karpenter and Amazon EC2 Spot Instances to improve its infrastructure management and cost optimization?"
top_chunks = get_top_k_documents(query=query, k=5)

Vamos ver quais `chunk`s nos devolveu

In [52]:
for i in range(len(top_chunks["ids"][0])):
    print(f"Rank {i+1}: {top_chunks['ids'][0][i]}, distance: {top_chunks['distances'][0][i]}")

Rank 1: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_0, distance: 0.29233667254447937
Rank 2: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_5, distance: 0.4007825255393982
Rank 3: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_1, distance: 0.4317566752433777
Rank 4: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_6, distance: 0.43832334876060486
Rank 5: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_4, distance: 0.44625571370124817


Como havia dito, o documento que tinha escolhido ao acaso era `Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt` e como pode ver os `chunk`s que nos devolveu são desse documento. Ou seja, de mais de 3000 `chunk`s existentes na base de dados, foi capaz de devolver os `chunk`s mais relevantes para essa pergunta, parece que isto funciona!

### Gerar a resposta

Como já temos os `chunk`s mais relevantes, passamos eles ao LLM, junto com a pergunta, para que este gere uma resposta

In [58]:
def generate_response(query, relevant_chunks, temperature=0.5, max_tokens=1024, top_p=0.7, stream=False):
    context = "\n\n".join([chunk for chunk in relevant_chunks])
    prompt = f"You are an assistant for question-answering. You have to answer the following question:\n\n{query}\n\nAnswer the question with the following information:\n\n{context}"
    message = [
        { "role": "user", "content": prompt }
    ]
    stream = client.chat.completions.create(
        messages=message, 
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stream=stream,
    )
    response = stream.choices[0].message.content
    return response

Testamos a função

In [59]:
response = generate_response(query, top_chunks["documents"][0])
print(response)

Neeva, a cloud-native, ad-free search engine founded in 2019, has leveraged Karpenter and Amazon EC2 Spot Instances to significantly improve its infrastructure management and cost optimization. Here’s how:

### Early Collaboration with Karpenter
In late 2021, Neeva began working closely with the Karpenter team, experimenting with and contributing fixes to an early version of Karpenter. This collaboration allowed Neeva to integrate Karpenter with its Kubernetes dashboard, enabling the company to gather valuable metrics on usage and performance.

### Combining Spot Instances and On-Demand Instances
Neeva runs its jobs on a large scale, which can lead to significant costs. To manage these costs effectively, the company adopted a combination of Amazon EC2 Spot Instances and On-Demand Instances. Spot Instances allow Neeva to bid on unused EC2 capacity, often at a fraction of the On-Demand price, while On-Demand Instances provide the necessary reliability for critical pipelines.

### Flexibi

Quando pedi ao LLM para gerar uma pergunta sobre o documento, também pedi para gerar a resposta correta. Esta é a resposta que o LLM me deu

```
Neeva used Karpenter and Amazon EC2 Spot Instances to improve its infrastructure management and cost optimization in several ways:

Simplified Instance Management:

Karpenter: By adopting Karpenter, Neeva simplified the process of provisioning and managing compute resources for its Amazon EKS clusters. Karpenter automatically provisions and de-provisions instances based on the workload, eliminating the need for manual configurations and reducing the complexity of understanding different compute instances.
Spot Instances: Neeva leveraged Amazon EC2 Spot Instances, which are unused EC2 capacity available at a significant discount (up to 90% cost savings). This allowed the company to control costs while meeting its performance requirements.
Enhanced Scalability:

Karpenter: Karpenter's ability to dynamically scale resources enabled Neeva to spin up new instances quickly, allowing the company to iterate at a higher velocity and run more experiments in less time.
Spot Instances: The use of Spot Instances provided flexibility and instance diversification, making it easier for Neeva to scale its compute resources efficiently.
Improved Productivity:

Karpenter: By democratizing infrastructure changes, Karpenter allowed any engineer to modify Kubernetes configurations, reducing the dependency on specialized expertise. This saved the Neeva team up to 100 hours per week of wait time on systems administration.
Spot Instances: The ability to quickly provision and de-provision Spot Instances reduced delays in the development pipeline, ensuring that jobs did not get stuck due to a lack of available resources.
Cost Efficiency:

Karpenter: Karpenter's best practices for Spot Instances, including flexibility and instance diversification, helped Neeva use these instances more effectively, staying within budget.
Spot Instances: The cost savings from using Spot Instances allowed Neeva to run large-scale jobs, such as indexing, for nearly the same cost but in a fraction of the time. For example, Neeva reduced its indexing jobs from 18 hours to just 3 hours.
Better Resource Utilization:

Karpenter: Karpenter provided better visibility into compute resource usage, allowing Neeva to track and optimize its resource consumption more closely.
Spot Instances: The combination of Karpenter and Spot Instances enabled Neeva to run large language models more efficiently, enhancing the search experience for its users.
In summary, Neeva's adoption of Karpenter and Amazon EC2 Spot Instances significantly improved its infrastructure management, cost optimization, and overall development efficiency, enabling the company to deliver better ad-free search experiences to its users.
```

E esta foi a resposta gerada pelo nosso `RAG`

```
Neeva, a cloud-native, ad-free search engine founded in 2019, has leveraged Karpenter and Amazon EC2 Spot Instances to significantly improve its infrastructure management and cost optimization. Here’s how:

### Early Collaboration with Karpenter
In late 2021, Neeva began working closely with the Karpenter team, experimenting with and contributing fixes to an early version of Karpenter. This collaboration allowed Neeva to integrate Karpenter with its Kubernetes dashboard, enabling the company to gather valuable metrics on usage and performance.

### Combining Spot Instances and On-Demand Instances
Neeva runs its jobs on a large scale, which can lead to significant costs. To manage these costs effectively, the company adopted a combination of Amazon EC2 Spot Instances and On-Demand Instances. Spot Instances allow Neeva to bid on unused EC2 capacity, often at a fraction of the On-Demand price, while On-Demand Instances provide the necessary reliability for critical pipelines.

### Flexibility and Instance Diversification
According to Mohit Agarwal, infrastructure engineering lead at Neeva, Karpenter's adoption of best practices for Spot Instances, including flexibility and instance diversification, has been crucial. This approach ensures that Neeva can dynamically adjust its compute resources to meet varying workloads while minimizing costs.

### Improved Scalability and Agility
By using Karpenter to provision infrastructure resources for its Amazon EKS clusters, Neeva has achieved several key benefits:
- **Scalability**: Neeva can scale its compute resources up or down as needed, ensuring that it always has the necessary capacity to handle its workloads.
- **Agility**: The company can iterate quickly and democratize infrastructure changes, reducing the time spent on systems administration by up to 100 hours per week.

### Enhanced Development Cycles
The integration of Karpenter and Spot Instances has also accelerated Neeva's development cycles. The company can now launch new features and improvements more rapidly, which is essential for maintaining a competitive edge in the search engine market.

### Cost Savings and Budget Control
Using Spot Instances, Neeva has been able to stay within its budget while meeting its performance requirements. This cost optimization is critical for a company that prioritizes user-first experiences and has no competing incentives from advertising.

### Future Plans
Neeva is committed to continuing its innovation and expansion. The company plans to launch in new regions and further improve its search engine, all while maintaining cost efficiency. As Mohit Agarwal notes, "The bulk of our compute is or will be managed using Karpenter going forward."

### Conclusion
By leveraging Karpenter and Amazon EC2 Spot Instances, Neeva has not only optimized its infrastructure costs but also enhanced its scalability, agility, and development speed. This strategic approach has positioned Neeva to deliver high-quality, ad-free search experiences to its users while maintaining a strong focus on cost control and innovation.
```

Por isso podemos concluir que o `RAG` funcionou corretamente!!!